In [1]:
just_checking_integrity=False

In [2]:
import gc
import pandas as pd
import scipy.sparse as sps

In [3]:
valid=sps.load_npz("../dataset/processed_data/URM_valid_bought.npz")
bought=valid.indices

In [5]:
dataset=None
dataset_max=None
#["UCF","ICBF","Graph","ICF","UCF_W","ICBF_W","Graph_W","ICF_W","UCF_WW","ICBF_WW","Graph_WW","ICF_WW"]
selected_list = ["UCF_W","ICBF_W","Graph_W","ICF_W"]
for file in selected_list:
    print(gc.collect())
    print(f"loading {file}")
    df=pd.read_csv(f"../dataset/candidates/traditional_recs/train/{file}.csv")
    print(len(df))
    print(f"loaded {file}")
    for col in df:
        if "Max" in col:
            df_max=df[["Session_Id",col]].drop_duplicates()
        elif "Score" in col:
            df_feats=df[["Session_Id","Item_ID",col]]
    if dataset is None:
        dataset=df_feats
        dataset_max=df_max
    else:
        dataset=dataset.merge(df_feats,on=["Session_Id","Item_ID"],how="outer")
        dataset_max=dataset_max.merge(df_max,on=["Session_Id"],how="outer")
    del df
        

#del dataset_max
del df_feats
del df_max

1930
loading UCF_W
18588456
loaded UCF_W
0
loading ICBF_W
16931962
loaded ICBF_W
0
loading Graph_W
9381571
loaded Graph_W
0
loading ICF_W
20341641
loaded ICF_W


In [8]:
#import ast
#submission_df=pd.read_csv("../dataset/candidates/NN/GRU/train/candidates.csv")
#if just_checking_integrity:
#    submission_df["score"]=submission_df["score"].apply(lambda x:x.replace("-inf","0")).apply(lambda x:x.replace("+inf","0")).apply(lambda x:x.replace("inf","0"))
#    submission_df["score"]=submission_df["score"].map(ast.literal_eval)
#else:
#    submission_df["score"]=submission_df["score"].map(ast.literal_eval)
#submission_df["code"]=submission_df["code"].map(ast.literal_eval)
#submission_df["rank"]=submission_df["rank"].map(ast.literal_eval)
#submission_df = submission_df.explode(['code','score', 'rank'])
#submission_df[["session_id","code","rank"]]=submission_df[["session_id","code","rank"]].astype(int)
#submission_df.rename(columns={"code":"Item_ID","session_id":"Session_Id","rank":"rank_gru","score":"score_gru"},inplace=True)

In [9]:
#submission_df.head()

In [6]:
import ast
submission_df_tr=pd.read_csv("../dataset/candidates/NN/Transformer/train/candidates.csv")
if just_checking_integrity:
    submission_df_tr["score"]=submission_df_tr["score"].apply(lambda x:x.replace("-inf","0"))
    submission_df_tr["score"]=submission_df_tr["score"].map(ast.literal_eval)
else:
    submission_df_tr["score"]=submission_df_tr["score"].map(ast.literal_eval)
submission_df_tr["code"]=submission_df_tr["code"].map(ast.literal_eval)
submission_df_tr["rank"]=submission_df_tr["rank"].map(ast.literal_eval)
submission_df_tr = submission_df_tr.explode(['code','score', 'rank'])
submission_df_tr[["session_id","code","rank"]]=submission_df_tr[["session_id","code","rank"]].astype(int)
submission_df_tr.rename(columns={"code":"Item_ID","session_id":"Session_Id","rank":"rank_transformer","score":"score_transformer"},inplace=True)

In [7]:
submission_df_tr.head()

,Session_Id,Item_ID,score_transformer,rank_transformer
0,918382,2785,0.061407,1
0,918382,3731,0.039482,2
0,918382,3355,0.023729,3
0,918382,1458,0.023356,4
0,918382,3820,0.019259,5


In [8]:
#import ast
#submission_df_LSTM=pd.read_csv("../dataset/candidates/NN/LSTM/train/candidates.csv")
#if just_checking_integrity:
#    submission_df_LSTM["score"]=submission_df_LSTM["score"].apply(lambda x:x.replace("-inf","0"))
#    submission_df_LSTM["score"]=submission_df_LSTM["score"].map(ast.literal_eval)
#else:
#    submission_df_LSTM["score"]=submission_df_LSTM["score"].map(ast.literal_eval)
#submission_df_LSTM["code"]=submission_df_LSTM["code"].map(ast.literal_eval)
#submission_df_LSTM["rank"]=submission_df_LSTM["rank"].map(ast.literal_eval)
#submission_df_LSTM = submission_df_LSTM.explode(['code','score', 'rank'])
#submission_df_LSTM[["session_id","code","rank"]]=submission_df_LSTM[["session_id","code","rank"]].astype(int)
#submission_df_LSTM.rename(columns={"code":"Item_ID","session_id":"Session_Id","rank":"rank_LSTM","score":"score_LSTM"},inplace=True)

In [10]:
#dataset=dataset.merge(submission_df,on=["Session_Id","Item_ID"],how="outer")
dataset=dataset.merge(submission_df_tr,on=["Session_Id","Item_ID"],how="outer")
#dataset=dataset.merge(submission_df_LSTM,on=["Session_Id","Item_ID"],how="outer")

dataset=dataset.merge(dataset_max,on=["Session_Id"],how="left")


In [11]:
if just_checking_integrity:
    dataset=dataset[:int(0.5*len(dataset))]

In [13]:
del dataset_max
#del submission_df
del submission_df_tr

NameError: name 'dataset_max' is not defined

In [14]:
import numpy as np
sessions=np.unique(dataset["Session_Id"])
train=pd.DataFrame()
train["Session_Id"]=sessions
if just_checking_integrity:
    train["Item_ID"]=bought[:len(sessions)]
else:
    train["Item_ID"]=bought
train["target"]=1

In [15]:
gc.collect()

4450

In [16]:
dataset=dataset.merge(train,on=["Session_Id","Item_ID"],how="left")
dataset.fillna(0,inplace=True)
gc.collect()

/var/folders/yt/h_zrk_k931s4wj_999xrfy0m0000gq/T/ipykernel_35562/1625650758.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dataset.fillna(0,inplace=True)


0

In [17]:
dataset["target"]=dataset["target"].astype("int8")
gc.collect()

0

In [18]:
sessions_containing_bought_among_candidates=dataset[["Session_Id","target"]].groupby("Session_Id").agg(sum).reset_index()
sessions_containing_bought_among_candidates=sessions_containing_bought_among_candidates.loc[sessions_containing_bought_among_candidates["target"]>0]

dataset=dataset[dataset['Session_Id'].isin(sessions_containing_bought_among_candidates["Session_Id"].tolist())]

/var/folders/yt/h_zrk_k931s4wj_999xrfy0m0000gq/T/ipykernel_35562/3285280766.py:1: FutureWarning: The provided callable <built-in function sum> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  sessions_containing_bought_among_candidates=dataset[["Session_Id","target"]].groupby("Session_Id").agg(sum).reset_index()


In [19]:
NHITS=len(dataset['Session_Id'].unique())
HITRATE=NHITS/81600 #approximate value 
HITRATE

0.7288235294117648

In [20]:
dataset.head(20)

,Session_Id,Item_ID,Score_UCF_W,Score_ICBF,Score_Graph_W,Score_ICF_W,score_transformer,rank_transformer,Max_Score_UCF_W,Max_Score_ICBF,Max_Score_Graph_W,Max_Score_ICF_W,target
525,918383,13,0.000000,0.000000,0.000000,0.000000,0.001024,88.0,594.02826,7.204127,91.79825,1.092114,0
526,918383,34,0.000000,0.181000,0.000000,0.000000,0.000000,0.0,594.02826,7.204127,91.79825,1.092114,0
527,918383,52,0.000000,0.015594,0.000000,0.000000,0.000000,0.0,594.02826,7.204127,91.79825,1.092114,0
528,918383,55,0.054177,0.632362,0.005355,0.018442,0.000000,0.0,594.02826,7.204127,91.79825,1.092114,0
529,918383,65,0.000000,0.274329,0.000000,0.000000,0.000000,0.0,594.02826,7.204127,91.79825,1.092114,0
530,918383,107,0.000000,0.000000,0.000000,0.005707,0.000000,0.0,594.02826,7.204127,91.79825,1.092114,0
531,918383,112,0.000000,0.045952,0.000000,0.000000,0.000000,0.0,594.02826,7.204127,91.79825,1.092114,0
532,918383,122,0.025767,0.000000,0.000000,0.000000,0.000000,0.0,594.02826,7.204127,91.79825,1.092114,0
533,918383,169,0.000000,0.839069,0.000000,0.000000,0.000000,0.0,594.02826,7.204127,91.79825,1.092114,0
534,918383,189,0.129413,0.000000,0.020602,0.278846,0.000000,0.0,594.02826,7.204127,91.79825,1.092114,0


In [22]:
dataset

,Session_Id,Item_ID,Score_UCF_W,Score_ICBF,Score_Graph_W,Score_ICF_W,score_transformer,rank_transformer,Max_Score_UCF_W,Max_Score_ICBF,Max_Score_Graph_W,Max_Score_ICF_W,target
525,918383,13,0.000000,0.000000,0.000000,0.000000,0.001024,88.0,594.028260,7.204127,91.798250,1.092114,0
526,918383,34,0.000000,0.181000,0.000000,0.000000,0.000000,0.0,594.028260,7.204127,91.798250,1.092114,0
527,918383,52,0.000000,0.015594,0.000000,0.000000,0.000000,0.0,594.028260,7.204127,91.798250,1.092114,0
528,918383,55,0.054177,0.632362,0.005355,0.018442,0.000000,0.0,594.028260,7.204127,91.798250,1.092114,0
529,918383,65,0.000000,0.274329,0.000000,0.000000,0.000000,0.0,594.028260,7.204127,91.798250,1.092114,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
39425260,999999,16357,0.000000,0.000000,0.000000,0.005289,0.000000,0.0,111.366875,9.421845,12.826448,0.786831,0
39425261,999999,16479,0.000000,0.000000,0.000000,0.541090,0.000000,0.0,111.366875,9.421845,12.826448,0.786831,0
39425262,999999,16520,0.000000,0.000000,0.000000,0.359696,0.000000,0.0,111.366875,9.421845,12.826448,0.786831,0
39425263,999999,18265,0.000000,0.000000,0.004365,0.000000,0.000000,0.0,111.366875,9.421845,12.826448,0.786831,0


In [23]:
dataset_macro_features = pd.read_csv('../dataset/processed_data/macro_feats_NN.csv',
                                     usecols=['session_id',
                                                'session_similarity',
                                                'session_similarity_uniques', 'session_similarity_feats', 'session_similarity_feats_uniques',
                                              'length_of_session_seconds',
                   'avg_time_spent_per_item_seconds',
                   'variance_time_spent_per_item_seconds',
                   'n_seen_items',
                   'n_unique_items',])

dataset_macro_features.rename(columns={'session_id':'Session_Id'}, inplace=True)
dataset = dataset.merge(dataset_macro_features, on='Session_Id', how='left')
del dataset_macro_features

In [24]:
#dataset.drop('Max_Score_TopPop15', axis=1, inplace=True)

In [25]:
from sklearn.model_selection import KFold
import lightgbm
import os
k=10


score=0
kf = KFold(n_splits=k)
index=0
for train_index, valid_index in kf.split(train):

    train_sessions=train["Session_Id"][train_index].tolist()
    valid_sessions=train["Session_Id"][valid_index].tolist()

    train_set=dataset[dataset["Session_Id"].isin(train_sessions)]
    valid_set=dataset[dataset["Session_Id"].isin(valid_sessions)]

    train_set=train_set.sort_values("Session_Id")
    valid_set=valid_set.sort_values("Session_Id")
    
    qids_train = train_set[["Session_Id"]].groupby("Session_Id")["Session_Id"].count().to_numpy()
    qids_valid = valid_set[["Session_Id"]].groupby("Session_Id")["Session_Id"].count().to_numpy()
    
    X_train=train_set.drop(["target","Session_Id","Item_ID"],axis=1)
    y_train=train_set["target"]
    
    X_valid=valid_set.drop(["target","Session_Id","Item_ID"],axis=1)
    y_valid=valid_set["target"]

    model = lightgbm.LGBMRanker(
    objective="lambdarank",
    #objective="rank_xendcg",
    #metric="mrr",
    n_estimators=2000,
    lambdarank_truncation_level=103,
    reg_alpha=1e-4,
    learning_rate=0.05,
    min_child_samples=400,
    subsample=0.8,
    subsample_freq=20
    )
    ranker=model.fit(
        X=X_train,
        y=y_train,
        group=qids_train,
        eval_set=[(X_valid, y_valid)],
        eval_group=[qids_valid],
        eval_at=(100),
        callbacks=[lightgbm.log_evaluation(),lightgbm.early_stopping(20,first_metric_only=True)],
        #verbose=10,
        eval_metric=["map","ndcg"]
    )
    ranker.booster_.save_model(f'../rankers/model_{index}.txt')
    #Get maximum val score
    score+=np.max(ranker._evals_result["valid_0"]["map@100"])
    index+=1

[LightGBM] [Info] Total groups: 53424, total data: 25497606
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 1.050983 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4330
[LightGBM] [Info] Number of data points in the train set: 25497606, number of used features: 19
[LightGBM] [Info] Total groups: 6048, total data: 2864949
[1]	valid_0's map@100: 0.127713	valid_0's ndcg@100: 0.251093
Training until validation scores don't improve for 20 rounds
[2]	valid_0's map@100: 0.217319	valid_0's ndcg@100: 0.324811
[3]	valid_0's map@100: 0.229011	valid_0's ndcg@100: 0.335358
[4]	valid_0's map@100: 0.231536	valid_0's ndcg@100: 0.337524
[5]	valid_0's map@100: 0.230708	valid_0's ndcg@100: 0.336708
[6]	valid_0's map@100: 0.232343	valid_0's ndcg@100: 0.33797
[7]	valid_0's map@100: 0.23173	valid_0's ndcg@100: 0.337748
[8]	valid_0's map@100: 0.232653	valid_0'

In [26]:
final_performcance=score/k
print("overall MAP@100 = ", final_performcance)

overall MAP@100 =  0.2567921520618986


In [27]:
print("normalized score ", final_performcance*HITRATE)

normalized score  0.18715616259099552
